In [1]:
import os
import json

sysparams = vars().copy()

### Specify the pipeline parameters

In [2]:
parameter_fn = 'parameters.json'

db_filename = "path/to/database/cosmos2.sqlite"
workflow_name = 'CARE_test'

base_dir = "path/to/the/directory/with/data"
input_dir = "input_data" # folder with the input data
data_dir = "CARE_data"  # folder to store CARE pairs
model_dir = 'CARE_models'
accuracy_dir = 'accuracy'
accuracy_fn = 'accuracy.csv'
n_jobs = 30
name_high='high'
name_low='low'
name_train='train'
name_validation='validation'
name_test='test'
axes = 'ZYX'
save_training_history = False

input_pairs = [('high_power/deconvolved', 'low_power/raw')]
test_fraction = 0.2
validation_fraction = 0.2
patch_size = [(16, 32, 32),
              (32, 64, 64)]
n_patches_per_image = [20]

train_epochs = 10
train_steps_per_epoch = [10]
train_batch_size = [8]


In [3]:
parameter_fn = os.path.abspath(parameter_fn)
args = vars().copy()
keys = [key for key in args.keys() 
        if not (key.startswith('_') or key in sysparams or key == 'sysparams')]
params = {key: args[key] for key in keys}
os.makedirs(os.path.dirname(parameter_fn), exist_ok=True)
with open(parameter_fn, 'w') as f:
    json.dump(params, f, indent=4)
params

{'parameter_fn': '/research/sharedresources/cbi/common/Anna/codes/COSMOS-CARE/parameters.json',
 'db_filename': 'path/to/database/cosmos2.sqlite',
 'workflow_name': 'CARE_test',
 'base_dir': 'path/to/the/directory/with/data',
 'input_dir': 'input_data',
 'data_dir': 'CARE_data',
 'model_dir': 'CARE_models',
 'accuracy_dir': 'accuracy',
 'accuracy_fn': 'accuracy.csv',
 'n_jobs': 30,
 'name_high': 'high',
 'name_low': 'low',
 'name_train': 'train',
 'name_validation': 'validation',
 'name_test': 'test',
 'axes': 'ZYX',
 'save_training_history': False,
 'input_pairs': [('high_power/deconvolved', 'low_power/raw')],
 'test_fraction': 0.2,
 'validation_fraction': 0.2,
 'patch_size': [(16, 32, 32), (32, 64, 64)],
 'n_patches_per_image': [20],
 'train_epochs': 10,
 'train_steps_per_epoch': [10],
 'train_batch_size': [8]}

### Running the workflow

To run locally, run `python cosmos_workflow.py -p parameters.json`

To generate a bsub command to submit to an lsf cluster, specify the following parameters:

In [4]:
n_cores = 30
n_gpus = 2
queue = 'dgx'
max_mem_gb = 300
python_path = "'/path/to/your/python/environment/bin/:$PATH'"

In [5]:
parameter_fn = os.path.abspath(parameter_fn)
code_path = os.path.abspath('cosmos_workflow.py')

In [6]:
command = rf'python {code_path} -p {parameter_fn} -g {n_gpus} -c {n_cores}'

command = f'bsub -P CARE -J CARE -q {queue} -n {n_cores} -gpu "num={n_gpus}:mode=exclusive_process"'\
          f' -R "rusage[mem={int(max_mem_gb/n_cores)}G]" "export PATH={python_path}; {command}"' 
print(command)

bsub -P CARE -J CARE -q dgx -n 30 -gpu "num=2:mode=exclusive_process" -R "rusage[mem=10G]" "export PATH='/path/to/your/python/environment/bin/:$PATH'; python /research/sharedresources/cbi/common/Anna/codes/COSMOS-CARE/cosmos_workflow.py -p /research/sharedresources/cbi/common/Anna/codes/COSMOS-CARE/parameters.json -g 2 -c 30"
